# Consumption report

After obtaining all the metrics of the best model, it is time to present a summary of the results obtained
with the best algorithm trained with a certain time and with a different interpolation method.

Note that all the results of the forecasts are saved in the path : C:/Users/USER/jupyter_notebooks/segundo_experimento_datos_acoplados_temporalmente-main/data/NILM/Results

Likewise, the results, in addition to presenting them in this notebook, are saved in:
c:/Users/USUAIO/jupyter_notebooks/segundo_experimento_datos_acoplados_temporalmente-main/data/results.htm

First, we need to load the libraries like plotly, nilmtk, pandas, datetime, and codecs.

In [1]:
import pandas as pd
import datetime
import codecs
from plotly.io import to_html
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px 
from plotly.subplots import make_subplots
from nilmtk import DataSet
import warnings
from IPython.display import SVG

warnings. filterwarnings("ignore")
plt.style.use('bmh')


## Brief explanation of the report function
We created a function that retrieves the predictions of the best model and calculates the total energy and power consumed on it.

It also offers other interesting data such as the start and end period, the sampling rate, etc.

We also obtained the graph of the disaggregated consumption of the application diffrenets compared to the reading of the main meter.

As relevant, we get all the interesting information, also in a file in html format, which will be overwritten as we execute this notebook.

In [ ]:
def report():
    # We take the best prediction, which for this second experiment of temporally synchronized data without taking harmonics is the following combination:
    # - CO algorithm (Combinatorial Optimization)
    # - Best sampling time (10 seconds)
    # - Best interpolation method (mean method)

    results = DataSet('C:/Users/carlo/jupyter_notebooks/open_multi_power_meter/data/NILM/Results/CO_30s_median_pred.h5')
    # Output file path
    html_dir = ('C:/Users/carlo/jupyter_notebooks/open_multi_power_meter/data/results.htm')

    labels = ['Main', 'Fan', 'Laptop Computer', 'Incandescent lamp', 'LED lamp', 'Television']  # labels

    results_elec = results.buildings[1].elec

    print(results_elec)  # print the different appliances with their associations

    # Pie chart
    df1 = pd.DataFrame(results_elec.submeters().fraction_per_meter(), columns=['Fraction'])
    df1['Energy'] = round(df1['Fraction'] * results_elec.total_energy()[0], 1)
    # df1['Device'] = labels

    fig1 = make_subplots(rows=1, cols=2, specs=[[{'type': 'domain'}, {'type': 'domain'}]])
    fig1.add_trace(go.Pie(values=df1['Fraction'], name='Percentage', hoverinfo="label+percent+name",
                          textinfo='label+percent', labels=labels, sort=False), 1, 1)
    fig1.add_trace(go.Pie(values=df1['Energy'], name='Energy', hoverinfo="label+value+name", textinfo='label+value',
                          labels=labels, sort=False, texttemplate="%{value:.1f} kWh"), 1, 2)
    fig1.update_traces(hole=.3)
    fig1.update_layout(width=900, height=450,
                       annotations=[dict(text='%', x=0.21, y=0.5, font_size=22, showarrow=False),
                                    dict(text='kWh', x=0.81, y=0.5, font_size=22, showarrow=False)])

    fig1.show()  # display the result on screen
    pie = fig1.to_html()  # convert the chart into an object to copy into an HTML file

    # Lines
    df2 = results_elec.select(instance=[2, 3, 4, 5, 6]).dataframe_of_meters(ac_type='active')
    df2.columns = labels[:5]
    fig2 = px.line(df2, width=900, height=500, labels={'variable': 'Devices', 'index': 'Date', 'value': 'Power (W)'})

    fig2.show()  # display the result on screen
    lines = fig2.to_html()  # convert the chart into an object to copy into an HTML file

    # Metadata
    date = results.metadata['date']  # retrieve date from dataset
    date = datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%S').strftime('%d/%m/%Y %H:%M:%S')

    sample_period = results.metadata['meter_devices']['mains']['sample_period']  # retrieve sampling period from dataset
    sample_period = str(sample_period) + ' seconds'
    print('Sample Period:', sample_period)

    start = results.metadata['timeframe']['start']  # retrieve start time from dataset
    start = datetime.datetime.strptime(start, '%Y-%m-%dT%H:%M:%S+01:00').strftime('%d/%m/%Y %H:%M:%S')
    print('Start:', start)

    end = results.metadata['timeframe']['end']  # retrieve end time from dataset
    end = datetime.datetime.strptime(end, '%Y-%m-%dT%H:%M:%S+01:00').strftime('%d/%m/%Y %H:%M:%S')
    print('End:', end)

    # Next, we create a static HTML string that includes the charts and the most relevant data retrieved from the dataset

    msg = """
    <html>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">
    <head>  <meta http-equiv="content-type" content="text/html; charset=UTF-8"> </head>
       <body>
        <h1><td><img src="logotipo.png" width="160" ></td><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Energy Consumption Disaggregation with OZM and NILMTK</b></h1>
          <br>  
        <h4><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Location and Date</b></h4>
      
        <table >
          <tbody>
            <tr>
            <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
             <td>Location:</td> <td>Electrotechnics Laboratory, Polytechnic School</td> </tr>
             <tr>
             <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
                        <td >Date:          </td>     <td >""" + date + """ </td>
            </tr>
          </tbody>
        </table>
        <h4><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Consumed Energy</b></h4>
        <table >
          <tbody>
            <tr>
            <td>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;</td>
              <td >Start Date:</td>
              <td >""" + start + """<br>
                </td>
            </tr>
            <tr>
            <td></td>
              <td >End Date:</td>
              <td >""" + end + """<br>
                </td>
            </tr>
            <tr>
            <td></td>
              <td >Sampling Period:</td>
              <td >""" + sample_period + """<br>
              </td>
            </tr>
          </tbody>
        </table>
        <p></p>
        <h4><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Disaggregated Energy Chart</b></h4>
        """ + pie + """
        <h4><b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Disaggregated Instantaneous Power Chart</b> </h4>
        """ + lines + """
        <p><br>
        </p>
        <p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; More information can be found at <a href="https://github.com/crn565/segundo_experimento_datos_acoplados_temporalmente">github.com/crn565</a></p>
       &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; October 2022 by CRN<span><br>
        </span><br>
        <br>
      </body>
     </html>"""

    with codecs.open(html_dir, 'w', encoding='utf8') as f:
        f.write(msg)     # write the msg string to the output file
        f.close()        # close the file
    return print('!The report has already been created!')

    
    

<h2>Configuration of MeterGroup</h2>
<pre>
MeterGroup(meters=
  <b>ElecMeter(instance=1, building=1, dataset='CO', site_meter, appliances=[])</b>
  <b>ElecMeter(instance=2, building=1, dataset='CO', appliances=[Appliance(type='fan', instance=1)])</b>
  <b>ElecMeter(instance=3, building=1, dataset='CO', appliances=[Appliance(type='laptop computer', instance=1)])</b>
  <b>ElecMeter(instance=4, building=1, dataset='CO', appliances=[Appliance(type='incandescent lamp', instance=1)])</b>
  <b>ElecMeter(instance=5, building=1, dataset='CO', appliances=[Appliance(type='LED lamp', instance=1)])</b>
  <b>ElecMeter(instance=6, building=1, dataset='CO', appliances=[Appliance(type='television', instance=1)])</b>
)
</pre>
<p><i>Calculating total_energy for ElecMeterID(instance=6, building=1, dataset='CO')...</i></p>


<img src="newplot.png" alt="informe" />

<img src="newplot2.png" alt="informe" />